# TeAAL specifications on variants of Loops' SpMV implementations (Work Oriented)

Description: All threads are assigned (`TOTAL_WORK` / `Number of Processors`) work items. Each thread then sequentially processes assigned work items in a loop. `TOTAL_WORK` = `NNZ` + `NUM_ROWS`.

During the Setup phase, the number of work units per thread is computed, and 2-D binary search within the nonzero indices and row offsets of a CSR matrix is conducted to get the starting position of each tile and atom in a thread-local variable. In the next phase, the algorithm builds the ranges for each thread to process as "complete" tiles and "partial" tiles. If a thread's atom range lies entirely within one tile, it is "complete", and is processed in a simple nested loop. If a thread's range crosses a tile boundary, the thread processes its work in a separate nested loop.

Template: https://github.com/gunrock/loops/blob/main/include/loops/algorithms/spmv/work_oriented.cuh

Scheduler (referenced as `config` below): https://github.com/gunrock/loops/blob/main/include/loops/schedule/work_oriented.hxx

GPU Kernel Template (Use it as a reference, don't execute it):

In [ ]:
'''
template <std::size_t threads_per_block,
          typename index_t,
          typename offset_t,
          typename type_t>
__global__ void __launch_bounds__(threads_per_block, 2)
    __work_oriented(std::size_t rows,
                    std::size_t cols,
                    std::size_t nnz,
                    offset_t* offsets,
                    index_t* indices,
                    const type_t* values,
                    const type_t* x,
                    type_t* y) {
  using setup_t =
      schedule::setup<schedule::algorithms_t::work_oriented, threads_per_block,
                      1, index_t, offset_t, std::size_t, std::size_t>;

  setup_t config(offsets, rows, nnz);
  auto map = config.init();

  /// Accumulate the complete tiles.
  type_t sum = 0;
  for (auto row : config.tiles(map)) {
    for (auto nz : config.atoms(row, map)) {
      sum += values[nz] * x[indices[nz]];
    }
    y[row] = sum;
    sum = 0;
  }

  // Interesting use of syncthreads to ensure all remaining tiles get processed
  // at the same time, possibly causing less thread divergence among the threads
  // in the same warp.
  __syncthreads();

  /// Process remaining tiles.
  for (auto row : config.remainder_tiles(map)) {
    for (auto nz : config.remainder_atoms(map)) {
      sum += values[nz] * x[indices[nz]];
    }
    /// Accumulate the remainder.
    if (sum != 0)
      atomicAdd(&(y[row]), sum);
  }
}
'''

## 2-D Binary Search (Not being used in TeAAL Specs)

In [ ]:
def setup(row_offsets, values, NUM_THREADS_PER_CYCLE, TOTAL_WORK, WORK_PER_THREAD):
    ranges = []
    for tid in range(NUM_THREADS_PER_CYCLE):
        upper = min((WORK_PER_THREAD * tid), TOTAL_WORK)
        lower = min((upper + WORK_PER_THREAD), TOTAL_WORK)

        st = search(upper, row_offsets[1:], values);
        en = search(lower, row_offsets[1:], values);
        ranges.append((st, en))
    return ranges

def search(diagonal, row_offsets, values):
    low = max((diagonal - len(values)), 0)
    high = min(diagonal, len(row_offsets))

    # binary search
    while low < high:
        mid = (low + high) // 2
        if row_offsets[mid] <= values[diagonal - mid - 1]:
            low = mid + 1
        else:
            high = mid

    return (min(low, len(row_offsets)), diagonal - low)

# Get CSR row offsets of A
A_NNZ = 0
row_offsets = [0]
values = []

row_it = iter(A_IJ.getRoot().getCoords())
val_it = iter(A_IJ.getRoot().getPayloads())
prev_row = next(row_it)

for row_id in range(I):
    if prev_row is not None:
        cur_row = prev_row
        if row_id == cur_row: # Check if empty row
            for nz in next(val_it).getPayloads():
                values.append(nz.v())
                A_NNZ += 1
            prev_row = next(row_it, None) # Returns None if iterator is done
    row_offsets.append(A_NNZ)

print("A row offsets:", row_offsets)
print("A values:", values)
print(f"A_NNZ: {A_NNZ}")

TOTAL_WORK = A_NNZ + I
WORK_PER_THREAD = math.ceil(TOTAL_WORK / NUM_THREADS_PER_CYCLE)
print(f"TOTAL_WORK: {TOTAL_WORK}, WORK_PER_THREAD = {WORK_PER_THREAD}")

# Get range for each tid
ranges = setup(row_offsets, values, NUM_THREADS_PER_CYCLE, TOTAL_WORK, WORK_PER_THREAD)

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "../..")

from src import utils

## Initialization

Initialize the input tensors.

For simplicity, suppose that each GPU SM processes 1 thread warp/block with size `BLOCK_SIZE` per cycle.

Things that are different from the actual implementation of the work-oriented method:
- Skipping the 2-D binary search to find the range of tiles. Therefore, `TOTAL_WORK` is simply equal to `A_NNZ`.
- Skipping the part of processing "complete" and "partial" tiles.

In [ ]:
I = 4
J = 4

NUM_SM = 2 # Number of GPU SMs 
BLOCK_SIZE = 2 # Number of threads per block 
NUM_THREADS_PER_CYCLE = BLOCK_SIZE * NUM_SM # Total number of threads processed per cycle

print(f"NUM_SM: {NUM_SM}, BLOCK_SIZE: {BLOCK_SIZE}, NUM_THREADS_PER_CYCLE: {NUM_THREADS_PER_CYCLE}")
seed = 1

A_IJ = Tensor.fromRandom(rank_ids=["I", "J"], shape=[I, J], seed=seed, density=[0.9, 0.6], name="A")
B_J = Tensor.fromRandom(rank_ids=["J"], shape=[J], seed=seed + 1, density=[1], name="B")

# Get A_NNZ
A_NNZ = 0

for row in A_IJ.getRoot().getPayloads():
    for nz in row.getPayloads():
        A_NNZ += 1

print(f"A_NNZ: {A_NNZ}")

TOTAL_WORK = A_NNZ
WORK_PER_THREAD = math.ceil(TOTAL_WORK / NUM_THREADS_PER_CYCLE)
print(f"TOTAL_WORK: {TOTAL_WORK}, WORK_PER_THREAD = {WORK_PER_THREAD}")

TeAAL Specifications:

In [ ]:
yaml = """
einsum:
  declaration:
    A: [I, J]
    B: [J]
    Z: [I]
  expressions:
    - Z[i] = A[i, j] * B[j]
mapping:
  rank-order:
    A: [I, J]
    B: [J]
    Z: [I]
  partitioning:
    Z:
      (I, J): [flatten()]
      IJ: [uniform_occupancy(A.WORK_PER_THREAD)]
  loop-order:
    Z: [IJ1, IJ0]
  spacetime:
    Z:
      space: [IJ1]
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

**Disclaimer**: The values for `Z_I` shown in the animation may be different from the actual values of `Z_I`. Run the command below to confirm if `Z_I` has been computed correctly.

In [ ]:
utils.check_matrix_vector_mul(A_IJ, B_J, Z_I)

## Performance on GPU

Load Balance: Nearly-perfect load balance across the threads. Threads are assigned to an equal number of work items. 

Assuming that the $A$ is stored in CSR format, $B$ and $Z$ are in uncompressed vectors, the memory access pattern would be:
- A: Uncoalesced access when threads are accessing complete tiles. There is an opportunity for coalesced access for partial tiles, but it is not guaranteed.
- B: Depends on the column indices of each nonzero entry of $A$. The more irregular the sparsity pattern that $A$ has, the more random the column indices of $A$'s nonzero entries will be. This should result in more uncoalesced accesses to $B$.
- Z: Same as $A$.